In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMRegressor

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/train.csv', parse_dates=['TIMESTAMP'])
test_df = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/test.csv', parse_dates=['TIMESTAMP'])

In [4]:
train_df.index = train_df['TIMESTAMP']
test_df.index = test_df['TIMESTAMP']

train_df = train_df.drop(columns=['TIMESTAMP'])
test_df = test_df.drop(columns=['TIMESTAMP'])

In [5]:
train_df.head()

,PRODUCT_ID,Y_Class,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,TRAIN_000,1,0.533433,T050304,A_31,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,TRAIN_001,2,0.541819,T050307,A_31,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,TRAIN_002,1,0.531267,T050304,A_31,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,TRAIN_003,2,0.537325,T050307,A_31,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,TRAIN_004,1,0.531590,T050304,A_31,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


In [6]:
test_df.head()

,PRODUCT_ID,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-09-09 02:01:00,TEST_000,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 02:09:00,TEST_001,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 08:42:00,TEST_002,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 10:56:00,TEST_003,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 11:04:00,TEST_004,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train_df = train_df.drop(columns=['PRODUCT_ID'])
test_df = test_df.drop(columns=['PRODUCT_ID'])

In [8]:
train_x = train_df.drop(columns=['Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

1. 데이터에 min-max scaling 적용

In [10]:
# origin train_x
train_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


In [22]:
df_minmax = train_x.iloc[:,2:]

print(type(df_minmax))

from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()

fitted = min_max_scaler.fit(df_minmax)

df_minmax = min_max_scaler.transform(df_minmax)

df_minmax = pd.DataFrame(df_minmax, columns = train_x.iloc[:,2:].columns, index = train_x.index)

<class 'pandas.core.frame.DataFrame'>


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [23]:
df_minmax.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.256757,0.248647,0.000000,0.122283,0.890487,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.240754,0.300866,0.407899,0.164742,0.601770,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.251422,0.133929,0.355835,0.205163,0.922566,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.199858,0.202110,0.704129,0.003057,0.559181,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.233997,0.275703,0.515978,0.088315,0.846239,NaN,NaN,NaN,NaN,NaN


In [25]:
train_x_minmax = pd.concat([train_x.iloc[:,0:2], df_minmax], axis=1)

train_x_minmax.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.256757,0.248647,0.000000,0.122283,0.890487,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.240754,0.300866,0.407899,0.164742,0.601770,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.251422,0.133929,0.355835,0.205163,0.922566,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.199858,0.202110,0.704129,0.003057,0.559181,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.233997,0.275703,0.515978,0.088315,0.846239,NaN,NaN,NaN,NaN,NaN


In [26]:
# 이후 scaling 진행한 데이터로 catboostclassifier 분류모델 학습

!pip install catboost
from catboost import CatBoostClassifier
cat = CatBoostClassifier(iterations=100, random_state=37)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.3 MB/s eta 0:00:00


In [27]:
cat.fit(train_x_minmax, train_y, cat_features=np.where(train_x.dtypes != np.float)[0])

<ipython-input-27-d35e68fa5c75>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cat.fit(train_x_minmax, train_y, cat_features=np.where(train_x.dtypes != np.float)[0])


Learning rate set to 0.5
0:	learn: 0.8305971	total: 961ms	remaining: 1m 35s
1:	learn: 0.7578487	total: 1.29s	remaining: 1m 3s
2:	learn: 0.7172675	total: 1.64s	remaining: 52.9s
3:	learn: 0.6909057	total: 1.98s	remaining: 47.5s
4:	learn: 0.6743387	total: 2.31s	remaining: 43.9s
5:	learn: 0.6561558	total: 2.65s	remaining: 41.5s
6:	learn: 0.6344991	total: 2.99s	remaining: 39.7s
7:	learn: 0.6130517	total: 3.32s	remaining: 38.2s
8:	learn: 0.5944930	total: 3.66s	remaining: 37s
9:	learn: 0.5864673	total: 4s	remaining: 36s
10:	learn: 0.5774144	total: 4.33s	remaining: 35s
11:	learn: 0.5643013	total: 4.66s	remaining: 34.2s
12:	learn: 0.5459619	total: 5s	remaining: 33.5s
13:	learn: 0.5385314	total: 5.35s	remaining: 32.8s
14:	learn: 0.5201931	total: 5.68s	remaining: 32.2s
15:	learn: 0.5062757	total: 6.03s	remaining: 31.7s
16:	learn: 0.4913925	total: 6.53s	remaining: 31.9s
17:	learn: 0.4821990	total: 7.11s	remaining: 32.4s
18:	learn: 0.4638384	total: 7.71s	remaining: 32.9s
19:	learn: 0.4540119	total:

In [28]:
# test데이터도 scaling 적용 후 넣어준다.

test_df.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-09-09 02:01:00,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 02:09:00,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 08:42:00,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 10:56:00,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 11:04:00,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
preds = cat.predict(test_df)

In [30]:
submit = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/sample_submission.csv')

submit['Y_Class'] = preds

In [31]:
submit.to_csv('/content/drive/MyDrive/LG Aimers 2기/제출/baseline_submission7.csv', index=False)

In [32]:
test_df_minmax = test_df.iloc[:,2:]

test_df_minmax = min_max_scaler.transform(test_df_minmax)

test_df_minmax = pd.DataFrame(test_df_minmax, columns = test_df.iloc[:,2:].columns, index = test_df.index)

In [34]:
test_minmax = pd.concat([test_df.iloc[:,0:2], test_df_minmax], axis=1)

test_minmax.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-09-09 02:01:00,T100306,T_31,0.009804,0.466667,0.0,0.0,0.0,0.0,0.352941,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 02:09:00,T100304,T_31,0.009804,0.400000,0.0,0.0,1.0,0.0,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 08:42:00,T100304,T_31,0.009804,0.533333,0.0,0.0,1.0,0.0,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 10:56:00,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 11:04:00,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
preds_2 = cat.predict(test_minmax)

submit = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/sample_submission.csv')

submit['Y_Class'] = preds

submit.to_csv('/content/drive/MyDrive/LG Aimers 2기/제출/baseline_submission8.csv', index=False)

In [36]:
submit = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/sample_submission.csv')

submit['Y_Class'] = preds_2

submit.to_csv('/content/drive/MyDrive/LG Aimers 2기/제출/baseline_submission9.csv', index=False)

# Robust Scaler

In [37]:
train_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


In [40]:
from sklearn.preprocessing import RobustScaler

train_robust = train_x.iloc[:,2:]

robust_scaler = RobustScaler()

fitted2 = robust_scaler.fit(train_robust)

train_robust = min_max_scaler.transform(train_robust)

train_robust = pd.DataFrame(train_robust, columns = train_x.iloc[:,2:].columns, index = train_x.index)

/usr/local/lib/python3.8/dist-packages/numpy/lib/nanfunctions.py:1119: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/usr/local/lib/python3.8/dist-packages/numpy/lib/nanfunctions.py:1374: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(


In [41]:
train_robust.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.256757,0.248647,0.000000,0.122283,0.890487,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.240754,0.300866,0.407899,0.164742,0.601770,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.251422,0.133929,0.355835,0.205163,0.922566,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.199858,0.202110,0.704129,0.003057,0.559181,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.233997,0.275703,0.515978,0.088315,0.846239,NaN,NaN,NaN,NaN,NaN


In [42]:
train_x_robust = pd.concat([train_x.iloc[:,0:2], train_robust], axis=1)

train_x_robust.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-06-13 05:14:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.256757,0.248647,0.000000,0.122283,0.890487,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:22:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.240754,0.300866,0.407899,0.164742,0.601770,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:30:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.251422,0.133929,0.355835,0.205163,0.922566,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:39:00,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.199858,0.202110,0.704129,0.003057,0.559181,NaN,NaN,NaN,NaN,NaN
2022-06-13 05:47:00,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.233997,0.275703,0.515978,0.088315,0.846239,NaN,NaN,NaN,NaN,NaN


In [43]:
cat2 = CatBoostClassifier(iterations=100, random_state=37)

cat2.fit(train_x_robust, train_y, cat_features=np.where(train_x_robust.dtypes != np.float)[0])

<ipython-input-43-7bce140c99e2>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cat2.fit(train_x_robust, train_y, cat_features=np.where(train_x_robust.dtypes != np.float)[0])


Learning rate set to 0.5
0:	learn: 0.8305971	total: 1.01s	remaining: 1m 39s
1:	learn: 0.7578487	total: 1.55s	remaining: 1m 15s
2:	learn: 0.7172675	total: 2.15s	remaining: 1m 9s
3:	learn: 0.6909057	total: 2.75s	remaining: 1m 5s
4:	learn: 0.6743387	total: 3.3s	remaining: 1m 2s
5:	learn: 0.6561558	total: 3.88s	remaining: 1m
6:	learn: 0.6344991	total: 4.51s	remaining: 59.9s
7:	learn: 0.6130517	total: 5.12s	remaining: 58.9s
8:	learn: 0.5944930	total: 5.51s	remaining: 55.7s
9:	learn: 0.5864673	total: 5.84s	remaining: 52.5s
10:	learn: 0.5774144	total: 6.18s	remaining: 50s
11:	learn: 0.5643013	total: 6.5s	remaining: 47.7s
12:	learn: 0.5459619	total: 6.84s	remaining: 45.8s
13:	learn: 0.5385314	total: 7.19s	remaining: 44.1s
14:	learn: 0.5201931	total: 7.53s	remaining: 42.7s
15:	learn: 0.5062757	total: 7.88s	remaining: 41.4s
16:	learn: 0.4913925	total: 8.21s	remaining: 40.1s
17:	learn: 0.4821990	total: 8.54s	remaining: 38.9s
18:	learn: 0.4638384	total: 8.88s	remaining: 37.8s
19:	learn: 0.4540119	

In [44]:
test_df.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-09-09 02:01:00,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 02:09:00,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 08:42:00,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 10:56:00,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 11:04:00,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
test_df_robust = test_df.iloc[:,2:]

test_df_robust = robust_scaler.transform(test_df_robust)

test_df_robust = pd.DataFrame(test_df_robust, columns = test_df.iloc[:,2:].columns, index = test_df.index)

In [46]:
test_robust = pd.concat([test_df.iloc[:,0:2], test_df_robust], axis=1)

test_robust.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2022-09-09 02:01:00,T100306,T_31,0.0,-0.2,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 02:09:00,T100304,T_31,0.0,-0.4,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 08:42:00,T100304,T_31,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 10:56:00,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-09 11:04:00,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
preds_3 = cat.predict(test_robust)

In [48]:
submit = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/sample_submission.csv')

submit['Y_Class'] = preds_3

submit.to_csv('/content/drive/MyDrive/LG Aimers 2기/제출/baseline_submission10.csv', index=False)